# MiSaTo-Dataset: a tutorial

In this notebook, we will show how our QM and MD dataset are stored in h5 files. We also show how the data can be loaded so that it can be used by a deep learning model. 

We start by importing the useful packages and set up the paths of the files

In [1]:
import h5py
import numpy as np 

import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

from data.components.datasets import MolDataset, ProtDataset
from data.components.transformQM import GNNTransformQM
from data.components.transformMD import GNNTransformMD
from data.qm_datamodule import QMDataModule
from data.md_datamodule import MDDataModule
from data.processing import preprocessing_db

In [2]:
qmh5_file = "../data/QM/h5_files/tiny_qm.hdf5"
norm_file = "../data/QM/h5_files/qm_norm.hdf5"
norm_txtfile = "../data/QM/splits/train_norm.txt"

## H5 files presentations

We read the QM H5 file and H5 file used to normalize the target values.

In [3]:
qm_H5File = h5py.File(qmh5_file)
qm_normFile = h5py.File(norm_file)

The ligands can be accessed using the pdb-id. Bellow we show the first ten molecules of the file.

In [4]:
qm_H5File.keys()

<KeysViewHDF5 ['10gs', '11gs', '13gs', '16pk', '184l', '185l', '186l', '187l', '188l', '1a07', '1a08', '1a09', '1a0q', '1a0tA', '1a0tB', '1a1b', '1a1c', '1a1e', '1a28', '1a2c', '1a30', '1a37', '1a42', '1a46', '1a4g', '1a4h', '1a4k', '1a4m', '1a4q', '1a4r', '1a4w', '1a50', '1a52', '1a5g', '1a5h', '1a5v', '1a61', '1a69', '1a7c', '1a7t', '1a7x', '1a85', '1a86', '1a8i', '1a8t', '1a94', '1a99', '1a9m', '1a9q', '1a9u']>

You can access to the molecule trajectories as follow

In [5]:
xyz = qm_H5File["10gs"]["atom_properties"]["atom_properties_values"][:, 0:3]

Target values can be accessed by specifiying into bracket the molecule name, then mol_properties and finally the name of the target value that we want to access: 

In [6]:
qm_H5File["10gs"]["mol_properties"]["Electron_Affinity"][()]

6.0974

We can access to the mean and standard-deviation of each target value by specifiying it into bracket.
We first specify the set, then the target value and finally either mean or std. 

In [7]:
qm_normFile.keys()

<KeysViewHDF5 ['Electron_Affinity', 'Electronegativity', 'Hardness', 'Ionization_Potential']>

In [8]:
print(qm_normFile["Electron_Affinity"]["mean"][()])
print(qm_normFile["Electron_Affinity"]["std"][()])

6.472403
3.8111687


## Datasets and dataloaders

### PyTorch

The QM and MD datasets are warped into a PyTorch Dataset class under the name MolDataset and ProtDataset, respectively. 
The parameters taken by the two classes as well as their types can be found as follow.

In [9]:
help(MolDataset)

Help on class MolDataset in module data.components.datasets:

class MolDataset(torch.utils.data.dataset.Dataset)
 |  MolDataset(data_file, idx_file, target_norm_file, transform, isTrain=False, post_transform=None)
 |  
 |  Load the QM dataset.
 |  
 |  Method resolution order:
 |      MolDataset
 |      torch.utils.data.dataset.Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, index: int)
 |  
 |  __init__(self, data_file, idx_file, target_norm_file, transform, isTrain=False, post_transform=None)
 |      Args:
 |          data_file (str): H5 file path
 |          idx_file (str): path of txt file which contains pdb ids for a specific split such as train, val or test.
 |          target_norm_file (str): H5 file path where training mean and std are stored.  
 |          transform (obj): class that convert a dict to a PyTorch Geometric graph.
 |          isTrain (bool, optional): Flag to standardize the target values (only used

In [10]:
help(ProtDataset)

Help on class ProtDataset in module data.components.datasets:

class ProtDataset(torch.utils.data.dataset.Dataset)
 |  ProtDataset(md_data_file, idx_file, transform=None, post_transform=None)
 |  
 |  Load the MD dataset
 |  
 |  Method resolution order:
 |      ProtDataset
 |      torch.utils.data.dataset.Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, index: int)
 |  
 |  __init__(self, md_data_file, idx_file, transform=None, post_transform=None)
 |      Args:
 |          md_data_file (str): H5 file path
 |          idx_file (str): path of txt file which contains pdb ids for a specific split such as train, val or test.
 |          transform (obj): class that convert a dict to a PyTorch Geometric graph.
 |          post_transform (PyTorch Geometric, optional): data augmentation. Defaults to None.
 |  
 |  __len__(self) -> int
 |  
 |  ----------------------------------------------------------------------
 |  Data and oth

We can load the data by instanciating MolDataset and providing the QM H5 file, the text file that indicates the molecule used for training and the norm file used to normalize the target values. 

The MolDataset class without any transform return a dictionary that contain the elements and their coordinates. We use GNNTransformQM class to transform our data to a graph that can be used by a GNN. The parameter post_transform is another transformation used to perform data augmentation.

In [11]:
train = "../data/QM/splits/train_tinyQM.txt"

transform = T.RandomTranslate(0.25)
batch_size = 128
num_workers = 48

data_train = MolDataset(qmh5_file, train, target_norm_file=norm_file, transform=GNNTransformQM(), post_transform=transform)

Finally, we can load our data using the PyTorch DataLoader.

In [12]:
train_loader = DataLoader(data_train, batch_size, shuffle=True, num_workers=0)

for idx, val in enumerate(train_loader):
    print(val)
    break

DataBatch(x=[1602, 25], edge_index=[2, 30354], edge_attr=[30354, 1], y=[60], pos=[1602, 3], id=[30], batch=[1602], ptr=[31])


### PyTorch lightning 

The QMDataModule is a class inherated from LightningDataModule that instanciate the MolDataset for training, validation and test set and retrun a dataloader for each set. 

We start by instanciation of the QMDataModule

In [13]:
files_root =  "../data/QM"

qmh5file = "h5_files/tiny_qm.hdf5"

tr = "splits/train_tinyQM.txt"
v = "splits/val_tinyQM.txt"
te = "splits/test_tinyQM.txt"

qmdata = QMDataModule(files_root, h5file=qmh5file, train=tr, val=v, test=te, num_workers=0)

Then, we call the setup function to instanciate the MolDataset for training, validation and test set

In [14]:
qmdata.setup()

Finally, we can return a dataloader for each set.

In [15]:
train_loader = qmdata.train_dataloader()

for idx, val in enumerate(train_loader):
    print(val)
    break
    

DataBatch(x=[1602, 25], edge_index=[2, 30354], edge_attr=[30354, 1], y=[60], pos=[1602, 3], id=[30], batch=[1602], ptr=[31])


# MD dataset

To generate the hdf5 file for the softness and hardness predictions

In [59]:
class Args:
    # input file
    datasetIn = "../data/MD/h5_files/tiny_md.hdf5"
    # Feature that should be stripped, e.g. atoms_element or atoms_type
    strip_feature = "atoms_element"
    # Value to strip, e.g. if strip_freature= atoms_element; 1 for H. 
    strip_value = 1
    # Start index of structures
    begin = 0
    # End index of structures
    end = 50 
    # We calculate the adaptability for each atom. 
    # Default behaviour will also strip H atoms, if no stripping should be perfomed set strip_value to -1.
    Adaptability = True
    # If set to True this will create a new feature that combines one entry for each protein AA but all ligand entries; 
    # e.g. for only ca set strip_feature = atoms_type and strip_value = 14
    Pres_Lat = False
    # We strip the complex by given distance (in Angstrom) from COG of molecule, 
    # use e.g. 15.0. If default value is given (0.0) no pocket stripping will be applied.
    Pocket = 0.0
    # output file name and location
    datasetOut = "../data/MD/h5_files/tiny_md_out.hdf5"


args = Args()

preprocessing_db.main(args)

10gs 1
{'trajectory_coordinates': <HDF5 dataset "trajectory_coordinates": shape (100, 6593, 3), type "<f8">, 'atoms_type': <HDF5 dataset "atoms_type": shape (6593,), type "<i8">, 'atoms_number': <HDF5 dataset "atoms_number": shape (6593,), type "<i8">, 'atoms_residue': <HDF5 dataset "atoms_residue": shape (6593,), type "<i8">, 'atoms_element': <HDF5 dataset "atoms_element": shape (6593,), type "<i8">, 'molecules_begin_atom_index': <HDF5 dataset "molecules_begin_atom_index": shape (3,), type "<i8">, 'frames_rmsd_ligand': <HDF5 dataset "frames_rmsd_ligand": shape (100,), type "<f8">, 'frames_distance': <HDF5 dataset "frames_distance": shape (100,), type "<f8">, 'frames_interaction_energy': <HDF5 dataset "frames_interaction_energy": shape (100,), type "<f8">, 'frames_bSASA': <HDF5 dataset "frames_bSASA": shape (100,), type "<f8">}
Stripping  atoms_element 1  and calculating adaptability for the atoms that were not stripped.
11gs 2
{'trajectory_coordinates': <HDF5 dataset "trajectory_coord

The same steps used for MD can be used to load the MD dataset

In [55]:
files_root =  ""

mdh5_file = '../data/MD/h5_files/tiny_md_out.hdf5'

train_idx = "../data/MD/splits/train_tiny_md.txt"
val_idx = "../data/MD/splits/val_tiny_md.txt"
test_idx = "../data/MD/splits/test_tiny_md.txt"

md_H5File = h5py.File(mdh5_file)

In [56]:
# Atom's coordinates from the first frame 
xyz = md_H5File['13gs']['trajectory_coordinates'][0, :, :] 

In [53]:
train_dataset = ProtDataset(mdh5_file, idx_file=train_idx, transform=GNNTransformMD(), post_transform=T.RandomTranslate(0.05))

train_loader = DataLoader(train_dataset, batch_size=16, num_workers=48)

In [54]:
for idx, val in enumerate(train_loader):
    print(val)
    break

DataBatch(x=[36840, 11], edge_index=[2, 603016], edge_attr=[603016], y=[36840], pos=[36840, 3], ids=[16], batch=[36840], ptr=[17])


In [50]:
mddata = MDDataModule(files_root, h5file=mdh5_file, train=train_idx, val=val_idx, test=test_idx, num_workers=0)

In [51]:
mddata.setup()

In [52]:
train_loader = mddata.train_dataloader()

for idx, val in enumerate(train_loader):
    print(val)
    break

DataBatch(x=[38378, 11], edge_index=[2, 622332], edge_attr=[622332], y=[38378], pos=[38378, 3], ids=[16], batch=[38378], ptr=[17])
